In [110]:
import pandas as pd

In [111]:
stats = pd.read_csv("FINAL.csv")

In [112]:
stats

,Unnamed: 0,Player,Age,G,GS,Cmp,pAtt,Cmp%,pYds,pTD,...,Year,rAtt,rYds,rTD,r1D,rLng,rY/A,rY/g,Fmb,Share
0,0,Kurt Warner,30.0,16.0,16.0,375.0,546.0,68.7,4830.0,36.0,...,2001,28.0,60.0,0.0,3.0,23.0,2.1,3.8,10.0,43.0
1,1,Peyton Manning,25.0,16.0,16.0,343.0,547.0,62.7,4131.0,26.0,...,2001,35.0,157.0,4.0,10.0,33.0,4.5,9.8,7.0,0.0
2,2,Brett Favre,32.0,16.0,16.0,314.0,510.0,61.6,3921.0,32.0,...,2001,38.0,56.0,1.0,6.0,14.0,1.5,3.5,16.0,10.0
3,3,Aaron Brooks,25.0,16.0,16.0,312.0,558.0,55.9,3832.0,26.0,...,2001,80.0,358.0,1.0,25.0,26.0,4.5,22.4,13.0,0.0
4,4,Rich Gannon,36.0,16.0,16.0,361.0,549.0,65.8,3828.0,27.0,...,2001,63.0,231.0,2.0,19.0,17.0,3.7,14.4,13.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1905,1905,Kadarius Toney,24.0,13.0,2.0,0.0,1.0,0.0,0.0,0.0,...,2023,11.0,31.0,0.0,2.0,14.0,2.8,2.4,1.0,0.0
1906,1906,Kyle Trask,25.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,...,2023,1.0,-1.0,0.0,0.0,-1.0,-1.0,-0.5,0.0,0.0
1907,1907,Garrett Wilson,23.0,17.0,17.0,0.0,1.0,0.0,0.0,0.0,...,2023,4.0,0.0,0.0,1.0,6.0,0.0,0.0,2.0,0.0
1908,1908,Christian Kirk,27.0,12.0,12.0,1.0,2.0,50.0,-1.0,0.0,...,2023,1.0,6.0,0.0,0.0,6.0,6.0,0.5,2.0,0.0


In [113]:
del stats["Unnamed: 0"]

In [116]:
#yippee, no nulls
#now we can load SKLearn's ridge linear regression ML model to extrapolate from our data

In [117]:
stats.columns

Index(['Player', 'Age', 'G', 'GS', 'Cmp', 'pAtt', 'Cmp%', 'pYds', 'pTD', 'TD%',
       'Int', 'Int%', 'p1D', 'pLng', 'pY/A', 'AY/A', 'Y/C', 'pY/g', 'Rate',
       'Sk', 'Yds-s', 'Sk%', 'NY/A', 'ANY/A', 'Year', 'rAtt', 'rYds', 'rTD',
       'r1D', 'rLng', 'rY/A', 'rY/g', 'Fmb', 'Share'],
      dtype='object')

In [118]:
#set predictors as parameters we want to consider
predictors = ['Age', 'G', 'GS', 'Cmp', 'pAtt', 'Cmp%', 'pYds', 'pTD', 'TD%',
       'Int', 'Int%', 'p1D', 'pLng', 'pY/A', 'AY/A', 'Y/C', 'pY/g', 'Rate',
       'Sk', 'Yds-s', 'Sk%', 'NY/A', 'ANY/A', 'Year', 'rAtt', 'rYds', 'rTD',
       'r1D', 'rLng', 'rY/A', 'rY/g', 'Fmb']

In [119]:
#NOTE: we did not include "Share" into our parameters.
#This is because we can't give the model information about what we want to predict (duh)

In [120]:
#now test-train split
#we use last year's data to determine how accurate the model is before predicting ahead
train = stats[(stats["Year"] < 2022)]
test = stats[stats["Year"] == 2022]

In [121]:
#import SKLearn
from sklearn.linear_model import Ridge

reg = Ridge(alpha=.1)

In [122]:
#now to fit model to data
reg.fit(train[predictors], train["Share"])

Ridge(alpha=0.1)

In [123]:
predictions = reg.predict(test[predictors])

In [124]:
#convert to Pandas DF
predictions = pd.DataFrame(predictions, columns=["Pred. Share"], index=test.index)

In [125]:
predictions

,Pred. Share
1721,16.772266
1722,-4.526593
1723,0.029187
1724,3.329952
1725,9.055485
...,...
1808,-3.081301
1809,-0.848585
1810,-0.178476
1811,-0.451775


In [126]:
#lets add player names to the DF to see who was predicted the highest
combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)

In [129]:
combination

,Player,Share,Pred. Share
1721,Patrick Mahomes,98.0,16.772266
1722,Justin Herbert,0.2,-4.526593
1723,Tom Brady,0.0,0.029187
1724,Kirk Cousins,0.0,3.329952
1725,Joe Burrow,25.6,9.055485
...,...,...,...
1808,Leonard Fournette,0.0,-3.081301
1809,Chad Henne,0.0,-0.848585
1810,Christian Kirk,0.0,-0.178476
1811,Cooper Kupp,0.0,-0.451775


In [130]:
#lets sort to see predicted winner
combination.sort_values("Share", ascending=False).head(10)

,Player,Share,Pred. Share
1721,Patrick Mahomes,98.0,16.772266
1730,Jalen Hurts,38.6,6.135958
1727,Josh Allen,30.2,13.776135
1725,Joe Burrow,25.6,9.055485
1795,Justin Jefferson,11.0,-0.055100
1729,Trevor Lawrence,0.6,4.945954
1722,Justin Herbert,0.2,-4.526593
1802,Derrick Henry,0.2,4.857235
1732,Tua Tagovailoa,0.2,6.535989
1746,Justin Fields,0.2,3.660907


In [131]:
#lets identify an error metric to determine how well the algorithm did
from sklearn.metrics import mean_squared_error

mean_squared_error(combination["Share"], combination["Pred. Share"])

97.27555083254246

In [132]:
combination["Share"].value_counts()

0.0     81
0.2      5
98.0     1
25.6     1
30.2     1
0.6      1
38.6     1
11.0     1
Name: Share, dtype: int64